In [1]:
!pip install backtesting

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from backtesting import Backtest, Strategy
import pandas as pd

In [18]:
tsla_data = pd.read_csv("data/TSLA.csv")
tsla_data["Date"] = pd.to_datetime(tsla_data["Date"])
tsla_data.set_index("Date", inplace = True)
# tsla_data = pd.Series(tsla_data["Adj Close"].values, index = tsla_data["Date"])

In [15]:
class MA_crossover_bt(Strategy):
    short_period = 10
    long_period = 50

    def init(self):
        self.short_period = short_period
        self.long_period = long_period
        # Inicializando as médias móveis: curta e longa
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)
    
    def moving_average(self, data, size):
        """Função para calcular a média móvel exponencial dos dados"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):
        # Certifique-se de que temos dados suficientes para realizar a operação
        if len(self.data) < max(self.short_period, self.long_period):
            return
        
        # Calculando a diferença e a variação da média curta
        diff = self.short_ma[-1] - self.long_ma[-1]
        short_average_var = self.short_ma[-1] - self.short_ma[-2] if len(self.short_ma) > 1 else 0

        # Condições de compra e venda adaptadas da sua estratégia:
        # Compra se a média curta está maior que a longa e subindo
        if diff > 0 and short_average_var > 0 and not self.position:
            self.buy()
        
        # Venda se a média curta está menor que a longa e caindo
        elif diff < 0 and short_average_var < 0 and self.position:
            self.sell()

In [19]:
# Parâmetros da estratégia
short_period = 10
long_period = 50

# Executando o backtest
bt = Backtest(tsla_data, MA_crossover_bt, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)
bt.plot()

Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                   98.511654
Equity Final [$]                1625937.45583
Equity Peak [$]                2993602.077736
Return [%]                       16159.374558
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                   43.374655
Volatility (Ann.) [%]               87.250031
Sharpe Ratio                         0.497131
Sortino Ratio                        1.177551
Calmar Ratio                         0.589072
Max. Drawdown [%]                   -73.63219
Avg. Drawdown [%]                   -9.737993
Max. Drawdown Duration     1021 days 00:00:00
Avg. Drawdown Duration       64 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                   16161.201475
Worst Trade [%]                  16161.201475
Avg. Trade [%]                   1

C:\Users\B51063\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\B51063\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\B51063\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\B51063\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1977', ...)